In [ ]:
# started 04/07/22
from mxnet import np, npx, autograd
npx.set_np()

In [ ]:
d = np.arange(20).reshape(4,5)
print(d)
# norms - > how big vertor is? L2 

In [ ]:
# example of gradient descent
x = np.arange(4)
# allocate memory for the gradient
x.attach_grad()
print(x.grad)
with autograd.record():
    y = 2 * np.dot(x, x)
y.backward() # calculate the gradient
print(x.grad)


In [ ]:
#  joint, marginal, and conditional probability

# conditional probabilities 
# P(A|B) = P(B|A) * P(A) / P(B)
# marginal probability // Відособлений розподіл
# P(B) = SUM ( P(A|B) ) 

# Joint Probability: Probability of events A and B.
# Marginal Probability: Probability of event X=A given variable Y.
# Conditional Probability: Probability of event A given event B.

In [ ]:
# analytical solution of loss function
# W = (XT X)-1 * XT * y (closed form / analytical solution )

# model the NN

In [ ]:
from d2l import mxnet as d2l
from mxnet import autograd, gluon, np, npx
from mxnet.gluon import nn
from mxnet import init
from mxnet import gluon
npx.set_np()

In [ ]:
true_w = np.array([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [ ]:
def load_array(data_arrays, batch_size, is_train=True): #@save
    """Construct a Gluon data iterator."""
    dataset = gluon.data.ArrayDataset(*data_arrays)
    return gluon.data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [ ]:
# model 
net = nn.Sequential()
net.add(nn.Dense(1))
# init W
net.initialize(init.Normal(sigma=0.01))
print(net)
# loss
loss = gluon.loss.L2Loss()
# SGD
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.01})

In [ ]:
# train
epoch = 3
for i in range(epoch):
    for data, label in data_iter:
        with autograd.record():
            l = loss(net(data), lable)
        l.backward()
        trainer.step(batch_size)
    print(f'epoch {epoch + 1}, loss {l.mean().asnumpy():f}')

In [ ]:
w = net[0].weight.data()
print(f'error in estimating w: {true_w - w.reshape(true_w.shape)}')
b = net[0].bias.data()
print(f'error in estimating b: {true_b - b}')

In [ ]:
# # soft max fucntion for classification 

In [ ]:
%matplotlib inline
from d2l import mxnet as d2l
from mxnet import gluon, init, npx
import mxnet as mx
import sys
from mxnet.gluon import nn
npx.set_np()
d2l.use_svg_display()

In [ ]:
mnist_train = mx.gluon.data.vision.datasets.FashionMNIST(train=True)
mnist_test = mx.gluon.data.vision.datasets.FashionMNIST(train=False)

In [ ]:
# `ToTensor` converts the image data from uint8 to 32-bit floating point. It
# divides all numbers by 255 so that all pixel values are between 0 and 1
transformer = gluon.data.vision.transforms.ToTensor() 

In [ ]:
batch_size = 256
train_iter = gluon.data.DataLoader(mnist_train.transform_first(transformer), batch_size=batch_size, shuffle=True, num_workers=8)
test_iter = gluon.data.DataLoader(mnist_test.transform_first(transformer), batch_size=batch_size, shuffle=False, num_workers=8)

In [ ]:
for x, y in train_iter:
    print(x.shape, y.shape)
    break

In [ ]:
# dataset has 10 classes; each image has 28 * 28 pixels size, so we got the vecor of size 784
number_inputs = 784
number_outputs = 10
# set the NN model
net = nn.Sequential()
net.add(nn.Dense(10)) # 10 classes 
net.initialize(init.Normal(sigma=0.01)) # set weights with intial values

In [ ]:
loss = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})

In [ ]:
num_epochs = 6
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer)

* Multilayer Perceptrons
* Linear Models May Go Wrong - so hidden layers increase complexity
* With deep neural networks, we used observational data to jointly learn both a representation via hidden 
* layers and a linear predictor that acts upon that representation.

* Incorporating Hidden Layers to Nonlinear relations 
* activation function
* MLPs are universal approximators
* ReLU Function
* sigmoid(x) = 1 / (1 + exp(−x))

In [ ]:
# Implement multilayer Perceptron
from d2l import mxnet as d2l
from mxnet import gluon, init, npx
import mxnet as mx
from mxnet.gluon import nn
npx.set_np()

In [ ]:
net = nn.Sequential()
net.add(nn.Dense(256, activation='relu'), nn.Dense(10)) # 256 units of 10 hidden layers with activation relu
net.initialize(init.Normal(sigma=0.01))
print(net.summary)

In [ ]:
d2l.train_ch3(
    net,
    train_iter,
    train_iter,
    gluon.loss.SoftmaxCrossEntropyLoss(), 
    10, # epoch
    gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})
    )

In [1]:
# Model complexity
## learn polynomial degrees 

## Prepare data
from d2l import mxnet as d2l
from mxnet import gluon, np, npx, init
from mxnet.gluon import nn
import math
npx.set_np()

max_degree = 20 # Maximum degree of the polynomial
n_train, n_test = 100, 100 # Training and test dataset sizes
true_w = np.zeros(max_degree) # Allocate lots of empty space
true_w[0:4] = np.array([5, 1.2, -3.4, 5.6])
features = np.random.normal(size=(n_train + n_test, 1))
np.random.shuffle(features)

poly_features = np.power(features, np.arange(max_degree).reshape(1, -1))
for i in range(max_degree):
    poly_features[:, i] /= math.gamma(i + 1) # `gamma(n)` = (n-1)!
    
# Shape of `labels`: (`n_train` + `n_test`,)
labels = np.dot(poly_features, true_w)
labels += np.random.normal(scale=0.1, size=labels.shape)

In [ ]:
def train(train_features, test_features, train_labels, test_labels, num_epochs=400):
     loss = gluon.loss.L2Loss()
     net = nn.Sequential()
     # Switch off the bias since we already catered for it in the polynomial features
     net.add(nn.Dense(1, use_bias=False))
     net.initialize()
     batch_size = min(10, train_labels.shape[0])
     train_iter = d2l.load_array((train_features, train_labels), batch_size)
     test_iter = d2l.load_array((test_features, test_labels), batch_size, is_train=False)
     trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})
     animator = d2l.Animator(xlabel='epoch', ylabel='loss', yscale='log', 
                    xlim=[1, num_epochs], ylim=[1e-3, 1e2], legend=['train', 'test'])
     for epoch in range(num_epochs):
          d2l.train_epoch_ch3(net, train_iter, loss, trainer)
          if epoch == 0 or (epoch + 1) % 20 == 0:
               animator.add(epoch + 1, (evaluate_loss(net, train_iter, loss), evaluate_loss(net, test_iter, loss)))
     print('weight:', net[0].weight.data().asnumpy())

In [ ]:
train(poly_features[:n_train, :4], poly_features[n_train:, :4],
            labels[:n_train], labels[n_train:])
# [5, 1.2, -3.4, 5.6]

In [ ]:
# Pick all the dimensions from the polynomial features
train(poly_features[:n_train, :], 
    poly_features[n_train:, :], 
    labels[:n_train], 
    labels[n_train:], num_epochs=1500)

In [ ]:
# Weight decay L2 - ridge regression algorithm / 
#  select models where features distributed more evenly / shrink the size of W towards 0

# L1 - lasso regression / lead to models with some set of features -> feature selection

In [ ]:
# DropOut 

## Naively applied, linear models do not take into account interactions among features.
#  For every feature, a linear model must assign either a positive or a negative weight, ignoring context.

# ... neural network overfitting is characterized by a state in which each layer relies on a specifc
# pattern of activations in the previous layer, calling this condition co-adaptation. Dropout, they
# claim, breaks up co-adaptation just as sexual reproduction is argued to break up co-adapted genes.

# Typically, we disable dropout at test time

In [ ]:
# J as the objective function

# Frobenius norm of the matrix is simply the L2 norm applied after flattening the matrix into a vector

In [1]:
# Backpropagation -> calculating the gradient

In [4]:
# 4.8 Numerical Stability and Initialization
## vamishing gradients 

In [24]:
# 5.2.1 Parameter Access
print(net[1].params)
print(net[1].bias)
print(net[1].bias.data())
net[1].weight.grad()
# all params
print(net[0].collect_params()) 
print(net.collect_params())

net.collect_params()['dense1_bias'].data()

In [ ]:
# Built-in Initialization
net.initialize(init=init.Normal(sigma=0.01), force_reinit=True)
net[0].weight.data()[0]

In [1]:
# 5.4.2 Layers with Parameters
from mxnet import gluon, np, npx, init
from mxnet.gluon import nn
npx.set_np()

class NNparametrixed(nn.Block):
    def __init__(self, input, output, **kwargs):
        super().__init__(**kwargs)
        self.ww = self.params.get('ww', shape=(output, input))
        self.bb = self.params.get('bb', shape=(input,))
    def forward(self, X):
        return npx.relu(
            np.dot(X, self.ww.data(ctx=X.ctx)) + self.bb.data(ctx=X.ctx)
        )
n = NNparametrixed(2, 2)
n.initialize()
n(np.random.uniform(size=(2, 2)))

array([[0.04363528, 0.09803008],
       [0.03606788, 0.08567319]])

In [3]:
from mxnet import np, npx
from mxnet.gluon import nn
npx.set_np()
npx.cpu(), npx.gpu(), npx.gpu(1)

(cpu(0), gpu(0), gpu(1))

In [2]:
npx.num_gpus()

0